[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/pooya-mohammadi/NER-Transformers-ArmanPers/blob/main/ArmanPers-NER.ipynb)

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
#! pip install datasets transformers seqeval

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [66]:
# from huggingface_hub import notebook_login

# notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [67]:
# !sudo apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [68]:
import transformers

print(transformers.__version__)

4.18.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/blob/main/examples/images/token_classification.png?raw=1)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [174]:
import os
import transformers
from datasets import load_metric
from armanpers_dataset import HFArmanPersDataset
from datasets import ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

In [70]:
model_checkpoint = "HooshvareLab/distilbert-fa-zwnj-base"
batch_size = 64

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

For our example here, we'll use the [ArmanPers](https://github.com/HaniehP/PersianNER). A custom dataset called ArmanPersDaset is created for this task

If you changed any of the text files and desire to reload them clear the cache address which in this case is `/home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0` because datasets does not check the files till the localfiles exists.

In [71]:
# !rm -rf /home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0
datasets = HFArmanPersDataset(train_path='train.txt',
                              val_path='val.txt',
                              test_path='test.txt').dataset

Reusing dataset arman_pers (/home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0)


Cache1 directory:  /home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0


  0%|          | 0/3 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [72]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 12290
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3074
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7682
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [74]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['B-event', 'B-fac', 'B-loc', 'B-org', 'B-pers', 'B-pro', 'I-event', 'I-fac', 'I-loc', 'I-org', 'I-pers', 'I-pro', 'O'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:

- "B-event": "رویداد",
- "B-fac": "امکانات",
- "B-loc": "موقعیت",
- "B-org": "سازمان",
- "B-pers": "شخص",
- "B-pro": "محصول",
- "I-event": "رویداد",
- "I-fac": "امکانات",
- "I-loc": "موقعیت",
- "I-org": "سازمان",
- "I-pers": "شخص",
- "I-pro": "محصول",
- "O": None

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [75]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['B-event',
 'B-fac',
 'B-loc',
 'B-org',
 'B-pers',
 'B-pro',
 'I-event',
 'I-fac',
 'I-loc',
 'I-org',
 'I-pers',
 'I-pro',
 'O']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [76]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [77]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,10476,"[آنچه, در, زیر, می‌آید, ،, خلاصه‌ای, از, دستاوردهای, اصلی, پوتین, در, يك, سال, گذشته, است, .]","[O, O, O, O, O, O, O, O, O, B-pers, O, O, O, O, O, O]"
1,1824,"["", عمل, نقد, "", نوشته, کاترین, بلزی, از, دیگر, کتابهای, آموزشی, در, زمینه, ادبیات, است, كه, عباس, مخبر, در, مجموعه, ترجمه‌های, خود, آن, را, به, فارسی, برگردانده, است, .]","[O, B-pro, I-pro, O, O, B-pers, I-pers, O, O, O, O, O, O, O, O, O, B-pers, I-pers, O, O, O, O, O, O, O, O, O, O, O]"
2,409,"[درخواست, تحقیق, و, تفحص, درباره, حادثه, خرم‌آباد, ،, تقدیم, مجلس, می‌شود, .]","[O, O, O, O, O, O, B-loc, O, O, O, O, O]"
3,12149,"[بنابراین, بر, خلاف, تصور, برخی, از, مخالفان, سطحی, خاتمی, ،, نظریه, گفتگوی, تمدنها, و, فرهنگها, نه, يك, نظریه, آمریکائی, بلکه, دقیقا, ضد, سلطه‌طلبی, و, تک‌قطبی‌گری, آمریکائی, است, .]","[O, O, O, O, O, O, O, O, B-pers, O, O, O, O, O, O, O, O, O, B-org, O, O, O, O, O, O, B-org, O, O]"
4,4506,"[0, ـ, رزرپین, :, در, موارد, پرفشاری, خون, و, بعضی, انواع, اختلالات, روانی, مصرف, می‌شود, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
5,4012,"[ضد, ارزش, بودن, اندیشه, تجزیه, و, تقسیم, اجتماع, نیز, در, همین, راستاست, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,3657,"[به, گفته, فرماندار, سراب, ،, سرمایه‌گذاران, می‌توانند, در, بخشهای, پرورش, ماهی, ،, دام, و, زنبور, عسل, ،, تولید, فرآورده‌های, لبنی, ،, جذب, گردشگر, و, معدن, این, شهرستان, سرمایه‌گذاری, کنند, .]","[O, O, O, B-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,2286,"[در, این, دیدار, ،, سید, حسین, هاشمی, ،, رئیس, گروه, دوستی, پارلمانی, ایران, و, کره, جنوبی, که, ریاست, هیأت, ایرانی, را, بر, عهده, دارد, ،, نیز, سیاستهای, تشنج‌زدایی, و, گفتگوی, تمدنها, را, تشریح, کرد, .]","[O, O, O, O, B-pers, I-pers, I-pers, O, O, O, O, O, B-org, O, B-org, I-org, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,12066,"[دکتر, لطفی, افزود, که, ترجمه‌های, فارسی, موجود, هنوز, به, هیچ, وجه, ،, حرف, آخر, در, ترجمه, آثار, ارسطو, نیست, ،, بلکه, باید, امیدوار, بود, که, در, آینده, ،, دیگران, نیز, بدین, کار, بپردازند, و, ترجمه‌های, بیشتری, پدید, آید, تا, جویندگان, دانش, با, مقایسه, این, ترجمه‌ها, که, حتما, باید, با, تحقیق, و, مراجعه, به, تفسیرها, همراه, باشد, ،, محتوای, فلسفی, این, کتابها, را, چنانکه, براستی, هست, ،, بشناسند, .]","[O, B-pers, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-pers, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,1679,"[هنرمندان, ایرانی, در, نقاشی, دوسالانه, معاصر, ایران, پنجمین, نمایشگاه, نقاشی, دوسالانه, معاصر, ایران, ،, از, امروز, در, موزه, هنرهای, معاصر, تهران, بر, پا, می‌شود, .]","[O, B-org, O, B-event, I-event, I-event, I-event, B-event, I-event, I-event, I-event, I-event, I-event, O, O, O, O, B-fac, I-fac, I-fac, I-fac, O, O, O, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [78]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/HooshvareLab/distilbert-fa-zwnj-base/resolve/main/config.json from cache at /home/ai/.cache/huggingface/transformers/448dd43db49d55aa2c264067c88728f01c15a91c25d470424fcabaea1500d779.03ba9625e6a5212fb76a3b52e259247b2de1a56115fcdbbc91363a663c7e1966
Model config DistilBertConfig {
  "_name_or_path": "HooshvareLab/distilbert-fa-zwnj-base",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.18.0",
  "vocab_size": 42000
}

loading file https://huggingface.co/HooshvareLab/distilbert-fa-zwnj-base/resolve/main/vocab.txt from cache at /h

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [79]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [80]:
print('sample: ', tokenizer("سلام این یک جمله است!"))

sample:  {'input_ids': [2, 3372, 1930, 1961, 2675, 1933, 110, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [81]:
print("tokenized sample: ",
      tokenizer(["سلام", ",", "این", "یک", "جمله", "تکه", "شده", "به", "کلمات", "است", "."], is_split_into_words=True))

tokenized sample:  {'input_ids': [2, 3372, 119, 1930, 1961, 2675, 6600, 1983, 1923, 5525, 1933, 121, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [82]:
example = datasets["train"][4]
print(example["tokens"])

['وي', 'افزود', ':', 'مافیای', 'قدرت', 'و', 'ثروت', '،', 'امروز', 'به', 'مقابله', 'جدی', 'با', 'جنبش', 'دانشجوئی', 'برخاسته\u200cاند', 'اما', 'طرفداران', 'سیاست', 'سرکوب', 'باید', 'این', 'واقعیت', 'را', 'درک', 'کنند', 'كه', 'لزوما', 'برنده', 'این', 'بحران\u200cآفرینیها', 'و', 'انسداد', 'سیاسی', 'نخواهند', 'بود', '.']


In [83]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'و', '##ي', 'افزود', ':', 'مافیای', 'قدرت', 'و', 'ثروت', '،', 'امروز', 'به', 'مقابله', 'جدی', 'با', 'جنبش', '[UNK]', 'برخاسته', '[ZWNJ]', 'اند', 'اما', 'طرفداران', 'سیاست', 'سرکوب', 'باید', 'این', 'واقعیت', 'را', 'درک', 'کنند', '[UNK]', 'لزوما', 'برنده', 'این', 'بحران', '[ZWNJ]', 'آ', '##فرین', '##یه', '##ا', 'و', 'انسداد', 'سیاسی', 'نخواهند', 'بود', '.', '[SEP]']


Here the words "وی" and "آفرینیها" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can use a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [84]:
len(example[f"ner_tags"]), len(tokenized_input["input_ids"])

(37, 47)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [85]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 30, 30, 30, 30, 30, 31, 32, 33, 34, 35, 36, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [86]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

47 47


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [87]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [88]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [89]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[2, 30060, 8573, 4979, 2748, 1921, 2240, 2690, 1925, 7936, 1969, 1927, 2531, 1923, 2898, 2626, 590, 2228, 133, 2129, 1925, 3615, 1, 2626, 623, 1, 1, 3359, 9, 4490, 2852, 2535, 590, 10149, 1993, 121, 3], [2, 3078, 1921, 3332, 1927, 3469, 18113, 595, 10808, 1916, 1923, 3468, 3721, 5129, 8429, 223, 3200, 209, 11738, 2139, 121, 3], [2, 2200, 2090, 2076, 5906, 5652, 12902, 595, 33624, 1114, 623, 1, 1921, 5046, 3400, 2818, 623, 1927, 1951, 4123, 6695, 121, 3], [2, 1930, 3570, 2299, 3886, 1925, 3570, 12573, 1933, 121, 3], [2, 623, 1173, 3218, 133, 27170, 2731, 623, 5484, 590, 2902, 1923, 4953, 4191, 1927, 6256, 1, 18419, 9, 2033, 2069, 5512, 3572, 7715, 2129, 1930, 3894, 1937, 3970, 2043, 1, 9204, 4691, 1930, 4890, 9, 595, 23988, 2195, 1110, 623, 16893, 3538, 7040, 1976, 121, 3]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [90]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0/cache-75bf32c739a59a0d.arrow
Loading cached processed dataset at /home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0/cache-eb525887db137b4d.arrow
Loading cached processed dataset at /home/ai/.cache/huggingface/datasets/arman_pers/ArmanPers/1.0.0/cache-9d792a2893c1c003.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [91]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

loading configuration file https://huggingface.co/HooshvareLab/distilbert-fa-zwnj-base/resolve/main/config.json from cache at /home/ai/.cache/huggingface/transformers/448dd43db49d55aa2c264067c88728f01c15a91c25d470424fcabaea1500d779.03ba9625e6a5212fb76a3b52e259247b2de1a56115fcdbbc91363a663c7e1966
Model config DistilBertConfig {
  "_name_or_path": "HooshvareLab/distilbert-fa-zwnj-base",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LA

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [172]:
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-finetuned-ner"
args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [99]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [100]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [101]:
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [102]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [103]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [104]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/ai/miniconda3/envs/torch_gpu/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12290
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1930


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.095497,0.710184,0.707774,0.708977,0.970442
2,No log,0.066796,0.787962,0.805281,0.796527,0.978986
3,0.139100,0.053783,0.840185,0.856173,0.848104,0.984164
4,0.139100,0.047197,0.861048,0.887594,0.874119,0.986866
5,0.139100,0.042697,0.892174,0.894675,0.893423,0.988830
6,0.029600,0.040122,0.899985,0.913262,0.906575,0.990047
7,0.029600,0.038046,0.911773,0.920785,0.916257,0.991012
8,0.013100,0.038597,0.921479,0.922702,0.922090,0.991532
9,0.013100,0.038304,0.921857,0.925800,0.923824,0.991666
10,0.013100,0.038131,0.920779,0.927570,0.924162,0.991666


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3074
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3074
  Batch size = 64
Saving model checkpoint to distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500
Configuration saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/config.json
Model weights saved in distilbert-fa-zwnj-base-finetuned-ner/checkpoint-500/pytorch_model.bin
to

TrainOutput(global_step=1930, training_loss=0.049034051326890066, metrics={'train_runtime': 263.5362, 'train_samples_per_second': 466.35, 'train_steps_per_second': 7.323, 'total_flos': 3985897424300340.0, 'train_loss': 0.049034051326890066, 'epoch': 10.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [105]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3074
  Batch size = 64


{'eval_loss': 0.038130514323711395,
 'eval_precision': 0.9207790306047737,
 'eval_recall': 0.9275704381177164,
 'eval_f1': 0.9241622574955908,
 'eval_accuracy': 0.9916664568594376,
 'eval_runtime': 3.1569,
 'eval_samples_per_second': 973.744,
 'eval_steps_per_second': 15.522,
 'epoch': 10.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [106]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3074
  Batch size = 64


{'event': {'precision': 0.78125,
  'recall': 0.8587786259541985,
  'f1': 0.8181818181818182,
  'number': 262},
 'fac': {'precision': 0.9353448275862069,
  'recall': 0.9730941704035875,
  'f1': 0.9538461538461539,
  'number': 223},
 'loc': {'precision': 0.9452789699570815,
  'recall': 0.9303062302006336,
  'f1': 0.9377328366152209,
  'number': 1894},
 'org': {'precision': 0.8862921348314606,
  'recall': 0.928436911487759,
  'f1': 0.906875143711198,
  'number': 2124},
 'pers': {'precision': 0.9637959407569939,
  'recall': 0.9648544755628775,
  'f1': 0.9643249176728869,
  'number': 1821},
 'pro': {'precision': 0.8942065491183879,
  'recall': 0.7802197802197802,
  'f1': 0.8333333333333333,
  'number': 455},
 'overall_precision': 0.9207790306047737,
 'overall_recall': 0.9275704381177164,
 'overall_f1': 0.9241622574955908,
 'overall_accuracy': 0.9916664568594376}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```

In [176]:
trainer.save_model(os.path.join(output_dir, "best"))

Saving model checkpoint to distilbert-fa-zwnj-base-finetuned-ner/best
Configuration saved in distilbert-fa-zwnj-base-finetuned-ner/best/config.json
Model weights saved in distilbert-fa-zwnj-base-finetuned-ner/best/pytorch_model.bin
tokenizer config file saved in distilbert-fa-zwnj-base-finetuned-ner/best/tokenizer_config.json
Special tokens file saved in distilbert-fa-zwnj-base-finetuned-ner/best/special_tokens_map.json


## Testing with trainer

In [259]:
example = "پویا محمدی در سازمان کشاورزی چه کاری میتواند بکند ؟"
tokens = tokenizer(example)
predictions, labels, _ = trainer.predict([tokens])
predictions = np.argmax(predictions, axis=2)
true_predictions = [[label_list[p] for p in prediction] for prediction in predictions]
for t, t_p in zip(tokenizer.decode(token_ids=tokens['input_ids']).split(" "), true_predictions[0]):
    print(t, " -> ", t_p)

***** Running Prediction *****
  Num examples = 1
  Batch size = 64


[CLS]  ->  O
پویا  ->  B-pers
محمدی  ->  I-pers
در  ->  O
سازمان  ->  B-org
کشاورزی  ->  I-org
چه  ->  O
کاری  ->  O
میتواند  ->  O
بکند  ->  O
؟  ->  O
[SEP]  ->  O


## Test

In [184]:
# load model and tokenizer
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer
import torch

device = "cuda"
model_type = "HooshvareLab/distilbert-fa-zwnj-base"
model_checkpoint = os.path.join(output_dir, "best")
tokenizer = AutoTokenizer.from_pretrained(model_type)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list)).to(device)

loading configuration file https://huggingface.co/HooshvareLab/distilbert-fa-zwnj-base/resolve/main/config.json from cache at /home/ai/.cache/huggingface/transformers/448dd43db49d55aa2c264067c88728f01c15a91c25d470424fcabaea1500d779.03ba9625e6a5212fb76a3b52e259247b2de1a56115fcdbbc91363a663c7e1966
Model config DistilBertConfig {
  "_name_or_path": "HooshvareLab/distilbert-fa-zwnj-base",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.18.0",
  "vocab_size": 42000
}

loading file https://huggingface.co/HooshvareLab/distilbert-fa-zwnj-base/resolve/main/vocab.txt from cache at /h

In [256]:
example = "پویا محمدی در سازمان کشاورزی چه کاری میتواند بکند ؟"
tokens = tokenizer(example)
input_ids = torch.tensor(tokens['input_ids']).unsqueeze(dim=0).to(device)
attention_mask = torch.tensor(tokens['attention_mask']).unsqueeze(dim=0).to(device)
with torch.no_grad():
    out = model(input_ids=input_ids, attention_mask=attention_mask)[0]
predictions = torch.argmax(out, axis=2).cpu()
true_predictions = [[label_list[p] for p in prediction] for prediction in predictions]
in_example = tokenizer.decode(token_ids=tokens['input_ids'])
for t, t_p in zip(in_example.split(" "), true_predictions[0]):
    print(t, " -> ", t_p)

_*:)*_